In [63]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import pandas as pd
from constants.Constants import PATH_ROUTES
from utils.CustomDataset import CustomDataset
from PIL import Image


In [65]:
print(PATH_ROUTES.PROJECT_ABSOLUTE_PATH)
train_index = pd.read_csv(PATH_ROUTES.TRAIN_CSV_INDEX_PATH)
train_dataset = CustomDataset(train_index=train_index, train_size=10)




c:\Users\bilal\OneDrive\Escritorio\Bilal\personal projects\lumbar-spine-degenerative-condition-classifier
